In [1]:
import itertools
import numpy as np
import math
from dismal import generator_matrices_ext
from scipy.stats import poisson
from dismal.generator_matrices import GeneratorMatrix
from dismal.likelihood_matrix import LikelihoodMatrix
from scipy import linalg

Define generator matrices

In [2]:
mig_mat1 = np.array(
    [
        [0, 0.1],
        [0.1, 0]
    ]
)


popsizes1 = [1, 1]

np.set_printoptions(precision=3)
q1 = generator_matrices_ext.q(mig_mat1, popsizes1)

In [3]:
mig_mat2 = np.array(
    [
        [0, 0],
        [0, 0]
    ]
)


popsizes2 = [1, 1]

np.set_printoptions(precision=3)
q2 = generator_matrices_ext.q(mig_mat2, popsizes2)

In [4]:
q2

array([[-1.,  0.,  0.,  1.],
       [ 0., -1.,  0.,  1.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [5]:
q3 = np.array(
    [[-1, 0, 0, 1],
    [0, -1, 0, 1],
    [0, 0, -1, 1],
    [0, 0, 0, 0]]
)

q3

array([[-1,  0,  0,  1],
       [ 0, -1,  0,  1],
       [ 0,  0, -1,  1],
       [ 0,  0,  0,  0]])

In [6]:
def matrix_multiplication(matrices):
    m = matrices[0]
    for i in range(1, len(matrices)):
        m = m @ matrices[i]
    
    return m

In [7]:
def alpha_matrix(eigenvals, s_vals, t_start, t_end):
    """Results are identical to old implementation"""

    eigenvals = np.array(eigenvals)
    s_vals = np.array(s_vals)

    alphas = [eigenvals/(eigenvals+1) * (1/(eigenvals+1))**s
              * np.exp(eigenvals * t_start)
              * (poisson.cdf(s, t_start * (eigenvals+1)) 
                 - poisson.cdf(s, t_end * (eigenvals+1))) for s in s_vals] 
    
    return np.transpose(alphas)

In [59]:
q1, q2 = GeneratorMatrix.from_params([1,1,1,1,1,1,1,0.1,0.1,0.1,0.1])

eigenvectors = [q1.eigenvectors, q2.eigenvectors]
eigenvalues = [-q1.eigenvalues[0:3], -q2.eigenvalues[0:3]]

In [63]:
def generalised_likelihood(g_matrices, eigenvals, p_matrices, gamma, t_vals, s_vals):

    n_matrices = len(g_matrices)
    assert n_matrices == len(eigenvals)
    assert n_matrices == len(p_matrices)

    likelihoods = []
    
    for i in [0,1,2]:
        most_recent = (g_matrices[0][i, 0:3] @ alpha_matrix(eigenvals[0], s_vals, t_start=0, t_end=t_vals[0])
                       + p_matrices[0][i, 0:3])

        middle = [g_matrices[n][0:3, 0:3] @ alpha_matrix(eigenvals[n], s_vals, t_start=t_vals[n-1], t_end=t_vals[n]) 
                   + (1-matrix_multiplication(p_matrices[0:n+1])[i,3]) for n in range(1, n_matrices)]
        likelihoods.append(np.array(most_recent) @ np.array(middle) * gamma)
    
    return -np.log(likelihoods)
    

Correct version is probably:
G[n] * lambda[n]
+ P[n] @ G[n+1] @ lambda[n+1]
+ [...]
+ (1-(G[n]@G[n+1]...)) * gamma

Trying first with working generator matrix implementation

In [64]:
p_matrices = [linalg.expm(q.matrix) for q in [q1, q2]]

In [65]:
generalised_likelihood(eigenvectors, eigenvalues, p_matrices, gamma=1, t_vals=[1,2], s_vals=np.array([[1,1,1], [1,1,1], [1,1,1]]))

array([[[1.46 , 1.46 , 1.46 ]],

       [[2.644, 2.644, 2.644]],

       [[0.261, 0.261, 0.261]]])

Comparison with existing DISMaL implementation

In [66]:
lm = LikelihoodMatrix(params={"theta0":1, "theta1":1, "theta2":1, "theta1_prime":1, "theta2_prime":1, "t1":1, "v":1, "M1_star":0.1, "M2_star":0.1, "M1_prime_star":0.1, "M2_prime_star": 0.1}, S=np.array([[1,1,1], [1,1,1], [1,1,1]]))

In [67]:
np.sum(np.sum((lm)))

[[0.728 1.418 2.06 ]
 [0.728 1.418 2.06 ]
 [2.355 1.689 1.525]]